# Evaluación de PC-SMOTE con Grid Search en el dataset Shuttle (Generación de caso base y datasets aumentados)


In [ ]:
# lo que hace es modificar la lista de rutas de búsqueda de módulos de Python (sys.path) para incluir las carpetas ../scripts y ../datasets como ubicaciones adicionales donde Python puede buscar módulos o paquetes cuando hacés un import.
import sys
sys.path.append("../scripts")
sys.path.append("../datasets")

## Importación de módulos y librerías necesarias


In [ ]:
# --- Módulos propios del proyecto ---
from cargar_dataset import cargar_dataset                      # Función para cargar datasets según configuración
from config_datasets import config_datasets                    # Diccionario de configuración de datasets
from evaluacion import evaluar_sampler_holdout                 # Evaluación de sobremuestreo con partición hold-out
from custom_samplers import PCSMOTEWrapper                     # Wrapper personalizado para la técnica PCSMOTE
from pc_smote import PCSMOTE                                   # Implementación principal de PCSMOTE
from graficador2d import Graficador2D                       # Clase para graficar resultados en 2D
from isolation_cleaner import IsolationCleaner                 # Clase para limpieza de outliers con Isolation Forest
from Utils import Utils                                       # Clase utilitaria con funciones auxiliares

# --- Librerías estándar de Python ---
from datetime import datetime, timedelta                       # Manejo de fechas y tiempos
from itertools import product                                  # Generación de combinaciones de parámetros
import os                                                      # Operaciones con el sistema de archivos
from pathlib import Path

import traceback
# --- Librerías científicas ---
import numpy as np                                              # Operaciones numéricas y algebra lineal
import pandas as pd                                             # Manipulación y análisis de datos tabulares
from scipy.stats import uniform                                 # Distribuciones para búsqueda de hiperparámetros

# --- Scikit-learn: preprocesamiento ---
from sklearn.preprocessing import LabelEncoder, StandardScaler # Codificación de etiquetas y escalado de datos
from sklearn.pipeline import make_pipeline, Pipeline            # Creación de pipelines de procesamiento y modelado

# --- Scikit-learn: división y validación ---
from sklearn.model_selection import (
    train_test_split,                                           # División de datos en train/test
    StratifiedKFold,                                            # Validación cruzada estratificada
    RandomizedSearchCV                                          # Búsqueda aleatoria de hiperparámetros
)

# --- Scikit-learn: reducción de dimensionalidad ---
from sklearn.decomposition import PCA                           # Análisis de Componentes Principales

# --- Scikit-learn: métricas ---
from sklearn.metrics import (
    f1_score,                                                    # Métrica F1-Score
    balanced_accuracy_score,                                     # Precisión balanceada
    matthews_corrcoef,                                           # Coeficiente MCC
    cohen_kappa_score,                                           # Kappa de Cohen
    make_scorer                                            
)

# --- Scikit-learn: clasificadores ---
from sklearn.ensemble import RandomForestClassifier             # Clasificador Random Forest
from sklearn.linear_model import LogisticRegression             # Regresión logística
from sklearn.svm import SVC                                      # Máquinas de Vectores de Soporte (SVM)

from sklearn.exceptions import ConvergenceWarning
import warnings


## Generación del caso base

Este código realiza dos tareas principales para cada dataset configurado en `config_datasets`:

1. **Generar el caso base** (subcarpeta `datasets_aumentados/base/`):
   - Se crea un directorio específico para almacenar la versión original del dataset sin ningún tipo de sobremuestreo.
   - El dataset se carga utilizando la misma función `cargar_dataset` empleada en el pipeline principal.
   - Si las etiquetas (`y`) están en formato de texto u objeto, se convierten a valores numéricos con `LabelEncoder`.
   - Se realiza una división estratificada en conjuntos de entrenamiento y prueba (`train/test`) utilizando `train_test_split` con una proporción 70/30 y una semilla fija para asegurar reproducibilidad.
   - Se guardan dos archivos CSV: `<nombre_dataset>_train.csv` y `<nombre_dataset>_test.csv`.

In [ ]:
# --- función: generar caso base (train/test sin sobremuestreo) ---

def generar_caso_base(
    nombre_dataset: str,
    config: dict,
    ruta_base: str = "../datasets/datasets_aumentados/base/",
    test_size: float = 0.30,
    random_state: int = 42,
    overwrite: bool = False
):
    """
    Genera el caso base (sin PCSMOTE) para un dataset: guarda train y test en ruta_base.
    Usa la misma lógica de carga que el resto del pipeline (cargar_dataset + LabelEncoder opcional).

    Retorna:
        (path_train, path_test)
    """
    os.makedirs(ruta_base, exist_ok=True)

    path_train = os.path.join(ruta_base, f"{nombre_dataset}_train.csv")
    path_test  = os.path.join(ruta_base, f"{nombre_dataset}_test.csv")

    if not overwrite and os.path.exists(path_train) and os.path.exists(path_test):
        return path_train, path_test  # ya generado

    # 1) Cargar dataset base con tu helper habitual
    X, y, _ = cargar_dataset(
        path=config["path"],
        clase_minoria=config.get("clase_minoria"),
        col_features=config.get("col_features"),
        col_target=config.get("col_target"),
        sep=config.get("sep", ","),
        header=config.get("header", None),
        binarizar=False,
        tipo=config.get("tipo", "tabular")
    )

    # 2) Asegurar etiquetas numéricas si vienen como strings/objects
    if getattr(y, "dtype", None) == object or (len(y) > 0 and isinstance(y[0], str)):
        y = LabelEncoder().fit_transform(y)

    # 3) Split estratificado (mismo seed para reproducibilidad)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, stratify=y, random_state=random_state
    )

    # 4) Guardar CSVs
    pd.concat([pd.DataFrame(X_train), pd.Series(y_train, name=config.get("col_target", "target"))], axis=1)\
      .to_csv(path_train, index=False)
    pd.concat([pd.DataFrame(X_test), pd.Series(y_test, name=config.get("col_target", "target"))], axis=1)\
      .to_csv(path_test, index=False)

    return path_train, path_test


In [ ]:
def aumentar_dataset_pcsmote_y_guardar(nombre_dataset, config, percentil_densidad, percentil_riesgo, criterio_pureza, test_size=0.2):
    print(f"📂 Cargando dataset: {nombre_dataset}")

    try:
        # 1) Cargar dataset original
        X, y, clases = cargar_dataset(
            path=config["path"],
            clase_minoria=config.get("clase_minoria"),
            col_features=config.get("col_features"),
            col_target=config.get("col_target"),
            sep=config.get("sep", ","),
            header=config.get("header", None),
            binarizar=False,
            tipo=config.get("tipo", "tabular"),
            impute='median',
            na_values=('?', 'NA', 'None')
        )

        # 2) Codificar etiquetas si son strings
        le = None
        if y.dtype == object or isinstance(y[0], str):
            le = LabelEncoder().fit(y)
            y = le.transform(y)

        # 3) Si es un dataset de imágenes, vectorizar
        if config.get("tipo") == "imagen":
            X = X.reshape((X.shape[0], -1)).astype(np.float32)

        # 4) Escalar TODO antes de dividir
        scaler = StandardScaler()
        X = scaler.fit_transform(X)

        # 5) Split estratificado
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=test_size, random_state=42, stratify=y
        )

        # 5.1) IsolationForest (opcional, por clase por defecto)
        X_train_clean, y_train_clean, info = IsolationCleaner.limpiarOutliers(
            X_train, y_train,
            contamination="auto",
            random_state=42,
            normalizar_scores=True,      # opcional; dejalo True si querés comparabilidad entre datasets
            percentil_umbral=5, # elimino el 5% más bajo de cada clase
            devolver_info=True,
            verbose=True
        )

        print("Umbrales aplicados:", info["umbral_por_clase"])

        # Sanity check pre-oversampling
        from collections import Counter
        print("📊 Train limpio (conteos):", Counter(y_train_clean))

        # 6) PCSMOTE sobre TRAIN LIMPIO
        print(f"🧬 Aplicando PCSMOTE | Densidad: {percentil_densidad} | Riesgo: {percentil_riesgo} | Pureza: {criterio_pureza}")
        sampler = PCSMOTE(
            random_state=42, #1 random_state
            percentil_densidad=percentil_densidad, #2 percentil_densidad
            percentil_entropia=75 if criterio_pureza == 'entropia' else None, #4 percentil_entropia
            percentil_riesgo = percentil_riesgo, # aproxima el criterio de Borderline-SMOTE de “≥ 0.5” 
            criterio_pureza=criterio_pureza, #5 criterio_pureza
            factor_equilibrio=config.get("factor_equilibrio", 1), #6 factor_equilibrio
            metric=config.get("metric", "euclidean") #7 metrica elegida
        )
        sampler.nombre_dataset = nombre_dataset

        # Descomentar si no quiero limpieza -- NO LIMPIEZA --
        # X_train_clean, y_train_clean = X_train, y_train

        if hasattr(sampler, "fit_resample_multiclass"):
            X_train_res, y_train_res = sampler.fit_resample_multiclass(X_train_clean, y_train_clean)
        else:
            X_train_res, y_train_res = sampler.fit_resample(X_train_clean, y_train_clean)

        print("📊 Train aumentado (conteos):", Counter(y_train_res))

        # 6.1) Plot: TRAIN LIMPIO vs TRAIN AUMENTADO (no X completo)
        idx_removed = info["idx_removed"]  # índices respecto de X_train

        grafo = Graficador2D(
            reductor="pca",
            semilla=42,
            percentil_densidad=percentil_densidad,
            percentil_riesgo=percentil_riesgo,
            criterio_pureza=criterio_pureza,
            nombre_dataset=nombre_dataset,
        )

        X_syn, y_syn = sampler.get_sinteticas()
        
        grafo.trazar_original_clean_aumentado(
            X_orig=X_train, y_orig=y_train,
            idx_removed=idx_removed,
            X_clean=X_train_clean, y_clean=y_train_clean,
            X_res=X_train_res, y_res=y_train_res,
            titulo="Train limpio vs. Train aumentado",
            nombres_clase=clases,
            X_syn=X_syn, y_syn=y_syn,
        )

        # 7) Guardar datasets: train aumentado
        print("💾 Guardando datasets aumentados...")

        out_dir = (Path.cwd().parent / "datasets" / "datasets_aumentados")
        out_dir.mkdir(parents=True, exist_ok=True)

        fname_train = f"pcsmote_{nombre_dataset}_D{percentil_densidad}_R{percentil_riesgo}_{Utils.tag_p(criterio_pureza)}_train.csv"
        fname_train = Utils.safe_token(fname_train)         
        p_train = (out_dir / fname_train)                   

        df_train = pd.DataFrame(X_train_res)
        df_train["target"] = y_train_res

        print(f"Verificacion de ruta p_train - Train aumentado: {p_train}")

        with open(p_train, "w", encoding="utf-8", newline="") as fh:
            df_train.to_csv(fh, index=False)

        # ---------- TEST (sin tocar) ----------
        fname_test = f"pcsmote_{nombre_dataset}_D{percentil_densidad}_R{percentil_riesgo}_{Utils.tag_p(criterio_pureza)}_test.csv"
        fname_test = Utils.safe_token(fname_test)
        p_test = (out_dir / fname_test)

        df_test = pd.DataFrame(X_test)
        df_test["target"] = y_test

        print(f"Verificacion de ruta p_test - Test original: {p_test}")
        with open(p_test, "w", encoding="utf-8", newline="") as fh:
            df_test.to_csv(fh, index=False)


        return str(p_train), sampler    


    except Exception as e:
        traceback.print_exc()
        print(f"❌ Error al aumentar dataset {nombre_dataset}: {e}")
        return None, None


### 🧬 Aumento de Datasets mediante Técnicas de Sobremuestreo

En esta etapa se genera una versión balanceada de cada dataset original mediante la aplicación de técnicas de sobremuestreo, con el objetivo de mitigar el desbalance de clases antes del entrenamiento de los modelos.

Actualmente, se emplea la técnica:

- `PCSMOTE` (Percentile-Controlled SMOTE), que permite controlar la generación de muestras sintéticas en función de percentiles de densidad, riesgo y pureza.

Para cada dataset, se exploran combinaciones específicas de parámetros según la técnica utilizada. Los datasets resultantes se almacenan en el directorio `datasets/datasets_aumentados/`, utilizando nombres de archivo que reflejan la configuración empleada (por ejemplo: `pcsmote_nombre_D25_R50_Pentropia_train.csv`).

> ⚠️ Esta fase no incluye entrenamiento ni validación de modelos. Su único propósito es generar conjuntos de datos aumentados a partir del conjunto de entrenamiento. La partición `train/test` se realiza previamente, y **solo la parte de entrenamiento es sometida a sobremuestreo**. El conjunto de prueba permanece sin modificar para garantizar una evaluación imparcial posterior.


In [ ]:
percentiles_densidad = [25, 50, 75]
percentiles_riesgo = [25, 50, 75]
criterios_pureza = ["entropia", "proporcion"]
combinaciones = list(product(percentiles_densidad, percentiles_riesgo, criterios_pureza))
os.makedirs("../logs/", exist_ok=True)

umbrales_por_dataset = {
    "shuttle": {"1": -0.108, "2": -0.446, "3":  0.006, "4":  0.125, "5":  0.138, "6": -0.779, "7": -0.921},
    "wdbc":    {"B": -0.029, "M": -0.264},
    "glass":   {"1": -0.859, "2": -0.319, "3": -0.982, "5": -0.429, "7": -0.824},
    "heart":   {"0": -0.470, "1": -0.742, "2": -0.824, "3": -0.552, "4": -0.727},
    "iris":    {"setosa": -0.776, "versicolor": -0.776, "virginica": -0.776},
    "ecoli":   {"imS": -0.315, "imL": -0.432, "imU": -0.289, "imT": -0.401, "imC": -0.567, "imW": -0.524, "imF": -0.486},
}

datasets_a_ignorar = {
        "eurosat", 
        "shuttle", 
        # "glass", 
        "heart", 
        "wdbc",
        "ecoli",
        "iris"
        }

for nombre_dataset, config in config_datasets.items():
    if nombre_dataset in datasets_a_ignorar:
        continue

    print(f"\n📁 Dataset: {nombre_dataset}")

    # --- CASO BASE ---
    base_train, base_test = generar_caso_base(nombre_dataset, config)
    print(f"🟦 Caso base generado:\n - Train: {base_train}\n - Test: {base_test}")

    # --- GRID DE PCSMOTE ---
    for idx, (pdens, priesgo, criterio) in enumerate(combinaciones, start=1):
        print(f"#{idx:02d} ➕ Aumentando con D={pdens} | R={priesgo} | P={criterio}")

        path_train, sampler = aumentar_dataset_pcsmote_y_guardar(
            nombre_dataset=nombre_dataset,
            config=config,
            percentil_densidad=pdens,
            percentil_riesgo=priesgo,
            criterio_pureza=criterio,
            # umbrales_por_dataset=umbrales_por_dataset,
        )

        if path_train and sampler:
            print(f"✅ Guardado exitoso:\n - Train: {path_train}")
            # --- rutas de logs (sanitizadas) ---
            base_logs = Path("../datasets/datasets_aumentados/logs/pcsmote")
            p_tag = Utils.tag_p(criterio)                          # "Pentropia"/"Pproporcion"
            fname_clase   = Utils.safe_token(f"log_pcsmote_{nombre_dataset}_D{pdens}_R{priesgo}_P{p_tag}.csv")
            fname_muestra = Utils.safe_token(f"log_pcsmote_x_muestra_{nombre_dataset}_D{pdens}_R{priesgo}_P{p_tag}.xlsx")

            log_path_x_clase    = str(base_logs / "por_clase"   / fname_clase)
            log_path_x_muestras = str(base_logs / "por_muestras"/ fname_muestra)

            # --- export logs (CSV/Excel) ---
            sampler.exportar_log_csv(log_path_x_clase)

            # si querés mantener la versión debug, OK; si no:
            sampler.exportar_log_muestras_excel(log_path_x_muestras)
            # sampler.exportar_log_muestras_excel_debug(log_path_x_muestras)

            print(f"📄 Log exportado: {log_path_x_clase}")
            print(f"📄 Log exportado: {log_path_x_muestras}")

        else:
            print("❌ Falló la generación.")
